In [2]:
!wget "ftp://ftp.inrialpes.fr/pub/lear/douze/data/jpg1.tar.gz"

--2021-01-03 06:58:34--  ftp://ftp.inrialpes.fr/pub/lear/douze/data/jpg1.tar.gz
           => ‘jpg1.tar.gz’
Resolving ftp.inrialpes.fr (ftp.inrialpes.fr)... 194.199.18.221
Connecting to ftp.inrialpes.fr (ftp.inrialpes.fr)|194.199.18.221|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/lear/douze/data ... done.
==> SIZE jpg1.tar.gz ... 1141827194
==> PASV ... done.    ==> RETR jpg1.tar.gz ... done.
Length: 1141827194 (1,1G) (unauthoritative)

jpg1.tar.gz         100%[===================>]   1,06G  3,80MB/s    in 7m 11s  

2021-01-03 07:05:48 (2,52 MB/s) - ‘jpg1.tar.gz’ saved [1141827194]



In [3]:
!wget "ftp://ftp.inrialpes.fr/pub/lear/douze/data/jpg2.tar.gz"

--2021-01-03 07:05:49--  ftp://ftp.inrialpes.fr/pub/lear/douze/data/jpg2.tar.gz
           => ‘jpg2.tar.gz’
Resolving ftp.inrialpes.fr (ftp.inrialpes.fr)... 194.199.18.221
Connecting to ftp.inrialpes.fr (ftp.inrialpes.fr)|194.199.18.221|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/lear/douze/data ... done.
==> SIZE jpg2.tar.gz ... 1701364177
==> PASV ... done.    ==> RETR jpg2.tar.gz ... done.
Length: 1701364177 (1,6G) (unauthoritative)

jpg2.tar.gz         100%[===================>]   1,58G  3,27MB/s    in 10m 24s 

2021-01-03 07:16:15 (2,60 MB/s) - ‘jpg2.tar.gz’ saved [1701364177]



In [ ]:
!tar -xf jpg1.tar.gz

In [ ]:
!tar -xf jpg2.tar.gz

In [7]:
import numpy as np
import torch 
from PIL import Image
from torchvision import transforms
import os
from tqdm.notebook import tqdm 
from skimage.feature import hog

ModuleNotFoundError: No module named 'torchvision'

# Feature Extractor

In [2]:
def feature_extractor(path_list, save_path, extractor_type):
    
    if type(path_list) != type([]):
        path_list = [path_list]
        
    if extractor_type == 'HOG':
        
        for file in tqdm(path_list):
            image = Image.open(file)
            image = image.resize((224,224))

            hog_vect = hog(image, orientations=16, pixels_per_cell=(8, 8),
                            cells_per_block=(1, 1), feature_vector=True, multichannel=True)

            vect_name = vect_name = os.path.join(save_path, 'hog_' + os.path.basename(file[:-4])) 
            #print(type(hog_vect))
            #print(hog_vect.shape)
            np.save(vect_name, hog_vect)
            
            
    elif extractor_type == 'CNN':
        
        model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True)
        model = torch.nn.Sequential(*(list(model.children())[:-1]))
        
        preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
         # create a mini-batch as expected by the model


        # move the input and model to GPU for speed if available
        if torch.cuda.is_available():
            input_batch = input_batch.to('cuda')
            model.to('cuda')

        with torch.no_grad():
            for file in tqdm(path_list):
                image = Image.open(file)
                image = image.resize((224,224))
                input_tensor = preprocess(image)
                input_batch = input_tensor.unsqueeze(0)

                output = model(input_batch)
                output = output.numpy().ravel()
                vect_name = os.path.join(save_path, 'cnn_' + os.path.basename(file[:-4])) 
                #print(type(hog_vect))
                #print(hog_vect.shape)
                np.save(vect_name, output)
                
    else: 
        print('Unknown extractor, please use either "HOB" or "CNN"')

In [3]:
!mkdir 'jpg_hog/'

mkdir: cannot create directory ‘jpg_hog/’: File exists


In [4]:
!mkdir 'jpg_cnn/'

mkdir: cannot create directory ‘jpg_cnn/’: File exists


In [6]:
file_paths = [os.path.join('jpg', file) for file in os.listdir('jpg')]

feature_extractor(file_paths, 'jpg_hog/', 'HOG')

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
file_paths = [os.path.join('jpg', file) for file in os.listdir('jpg')]

feature_extractor(file_paths, 'jpg_hog/', 'HOG')